In [ ]:
import os
import requests

In [20]:
BASE_URL = "https://mdhopendata.blob.core.windows.net/verkehrsdetektion/{year}/alte_qualitaetssicherung/Fahrstreifendetektoren/det_val_hr_{year}_{month}.csv.gz"
ALT_URL = "https://mdhopendata.blob.core.windows.net/verkehrsdetektion/{year}/Detektoren%20(einzelne%20Fahrspur)/det_val_hr_{year}_{month}.csv.gz"

OUTPUT_DIR = "detectors_data_raw"

START_YEAR = 2015
END_YEAR = 2024         # 2025 is not up to date yet

os.makedirs(OUTPUT_DIR, exist_ok=True)

In [21]:
years_using_alt = set()   # store years where ALT_URL was used successfully
month_failed = set()     # store months that failed to download

def try_download(url, out_path):
    """Try to download URL. Return True if file is downloaded."""
    try:
        r = requests.get(url, timeout=20)
        if r.status_code == 200 and len(r.content) > 1000:  # avoid empty files
            with open(out_path, "wb") as f:
                f.write(r.content)
            return True
        return False
    except Exception:
        return False

In [22]:
# DOWNLOAD LOOP

for year in range(START_YEAR, END_YEAR + 1):
    year_dir = os.path.join(OUTPUT_DIR, str(year))
    os.makedirs(year_dir, exist_ok=True)

    print(f"\n=== YEAR {year} ===")

    for month in range(1, 13):
        month_str = str(month).zfill(2)

        out_path = os.path.join(year_dir, f"detections_{year}_{month_str}.csv.gz")

        # Build URLs
        url_main = BASE_URL.format(year=year, month=month_str)
        url_alt  = ALT_URL.format(year=year, month=month_str)

        # Try main
        print(f"Trying MAIN: {url_main}")
        if try_download(url_main, out_path):
            print(f" → OK (MAIN)")
            continue

        # Try alternative
        print(f" MAIN failed → trying ALT: {url_alt}")
        if try_download(url_alt, out_path):
            print(f" → OK (ALT)")
            years_using_alt.add(year)
        else:
            print(f" → FAILED (both main and alt) → skipping")
            month_failed.add(str(year) + month_str)



=== YEAR 2015 ===
Trying MAIN: https://mdhopendata.blob.core.windows.net/verkehrsdetektion/2015/alte_qualitaetssicherung/Fahrstreifendetektoren/det_val_hr_2015_01.csv.gz
 → OK (MAIN)
Trying MAIN: https://mdhopendata.blob.core.windows.net/verkehrsdetektion/2015/alte_qualitaetssicherung/Fahrstreifendetektoren/det_val_hr_2015_02.csv.gz
 → OK (MAIN)
Trying MAIN: https://mdhopendata.blob.core.windows.net/verkehrsdetektion/2015/alte_qualitaetssicherung/Fahrstreifendetektoren/det_val_hr_2015_03.csv.gz
 → OK (MAIN)
Trying MAIN: https://mdhopendata.blob.core.windows.net/verkehrsdetektion/2015/alte_qualitaetssicherung/Fahrstreifendetektoren/det_val_hr_2015_04.csv.gz
 MAIN failed → trying ALT: https://mdhopendata.blob.core.windows.net/verkehrsdetektion/2015/Detektoren%20(einzelne%20Fahrspur)/det_val_hr_2015_04.csv.gz
 → FAILED (both main and alt) → skipping
Trying MAIN: https://mdhopendata.blob.core.windows.net/verkehrsdetektion/2015/alte_qualitaetssicherung/Fahrstreifendetektoren/det_val_hr_201

In [23]:
# SUMMARY

print("\n===============================")
print("DOWNLOAD COMPLETE.")
print("Years where ALT_URL was used:")
print(sorted(list(years_using_alt)))
print("Months that failed to download:")
print(sorted(list(month_failed)))
print("===============================")


DOWNLOAD COMPLETE.
Years where ALT_URL was used:
[2018, 2019, 2020, 2021]
Months that failed to download:
['201504', '201610', '201710', '201711', '201712', '202203', '202204', '202210']


In [ ]:
import pandas as pd

DATA_DIR = OUTPUT_DIR
FLAGGED_YEARS = sorted(list(years_using_alt))

def load_any_csv_in_year(year):
    """Load the first few rows of the first CSV.GZ in a given year."""
    year_dir = os.path.join(DATA_DIR, str(year))
    if not os.path.isdir(year_dir):
        return None
    
    for fname in os.listdir(year_dir):
        if fname.endswith(".csv.gz"):
            path = os.path.join(year_dir, fname)
            # Pandas can load gzip-compressed CSV directly
            try:
                df = pd.read_csv(path, sep=";", nrows=5)
                return df
            except Exception as e:
                print(f"Error loading {path}: {e}")
                return None

    return None


# ---------------------------------------------------------
# 1. Find REFERENCE FORMAT (first non-flagged year)
# ---------------------------------------------------------
ref_df = None
ref_year = None

for year in range(START_YEAR, END_YEAR + 1):
    df = load_any_csv_in_year(year)
    if df is not None:
        ref_df = df
        ref_year = year
        break

if ref_df is None:
    print("Error: no reference CSV found.")
else:
    print(f"Reference format loaded from year {ref_year}")
    print("Reference columns:", ref_df.columns.tolist())


# ---------------------------------------------------------
# 2. Compare all FLAGGED years to the reference
# ---------------------------------------------------------
differences = []

for year in FLAGGED_YEARS:
    print(f"\nChecking flagged year {year}...")
    df = load_any_csv_in_year(year)

    if df is None:
        print(" → No CSV found or could not load.")
        continue

    cols = df.columns.tolist()
    print(" → Columns:", cols)

    if cols != ref_df.columns.tolist():
        print(" *** FORMAT DIFFERENCE DETECTED ***")
        differences.append((year, cols))


# ---------------------------------------------------------
# 3. Summary
# ---------------------------------------------------------
if differences:
    print("\n====== DIFFERENT FORMAT FOUND ======")
    for year, cols in differences:
        print(f"Year {year} differs:")
        print(cols)
else:
    print("\n====== ALL FLAGGED YEARS MATCH THE BASE FORMAT ======")


Reference format loaded from year 2015
Reference columns: ['detid_15', 'tag', 'stunde', 'qualitaet', 'q_kfz_det_hr', 'v_kfz_det_hr', 'q_pkw_det_hr', 'v_pkw_det_hr', 'q_lkw_det_hr', 'v_lkw_det_hr']

Checking flagged year 2018...
 → Columns: ['detid_15', 'tag', 'stunde', 'qualitaet', 'q_kfz_det_hr', 'v_kfz_det_hr', 'q_pkw_det_hr', 'v_pkw_det_hr', 'q_lkw_det_hr', 'v_lkw_det_hr']

Checking flagged year 2019...
 → Columns: ['detid_15', 'tag', 'stunde', 'qualitaet', 'q_kfz_det_hr', 'v_kfz_det_hr', 'q_pkw_det_hr', 'v_pkw_det_hr', 'q_lkw_det_hr', 'v_lkw_det_hr']

Checking flagged year 2020...
 → Columns: ['detid_15', 'tag', 'stunde', 'qualitaet', 'q_kfz_det_hr', 'v_kfz_det_hr', 'q_pkw_det_hr', 'v_pkw_det_hr', 'q_lkw_det_hr', 'v_lkw_det_hr']

Checking flagged year 2021...
 → Columns: ['detid_15', 'tag', 'stunde', 'qualitaet', 'q_kfz_det_hr', 'v_kfz_det_hr', 'q_pkw_det_hr', 'v_pkw_det_hr', 'q_lkw_det_hr', 'v_lkw_det_hr']

====== ALL FLAGGED YEARS MATCH THE BASE FORMAT ======


In [ ]:
df = pd.read_csv(os.path.join(OUTPUT_DIR, "2015", "detections_2015_01.csv.gz"), sep=";")

df.head()

In [28]:
import os
import pandas as pd

DATA_DIR = OUTPUT_DIR
OUTPUT = os.path.join(DATA_DIR, "raw_traffic.csv")

# Columns in the TEU detector files
COLUMNS = [
    'detid_15',         # detector ID
    'tag',              # date (YYYYMMDD)
    'stunde',           # hour (0–23)
    'qualitaet',        # quality flag
    'q_kfz_det_hr',     # vehicle count
    'v_kfz_det_hr',     # avg speed
    'q_pkw_det_hr',     # car count
    'v_pkw_det_hr',     # car speed
    'q_lkw_det_hr',     # truck count
    'v_lkw_det_hr'      # truck speed
]

# list to store all pieces before concatenation
frames = []

print("Aggregating detector data...\n")

for year in range(2015, 2026):
    year_dir = os.path.join(DATA_DIR, str(year))
    if not os.path.isdir(year_dir):
        continue

    print(f"=== YEAR {year} ===")

    for fname in os.listdir(year_dir):
        if not fname.endswith(".csv.gz"):
            continue
        
        path = os.path.join(year_dir, fname)
        
        try:
            df = pd.read_csv(path, sep=";", usecols=COLUMNS)
        except Exception as e:
            print(f"Skipping {path}: {e}")
            continue
        
        # Build datetime column
        df["timestamp"] = pd.to_datetime(
            df["tag"].astype(str) + df["stunde"].astype(str).str.zfill(2),
            format="%d.%m.%Y%H"
        )
        
        # add metadata (useful for debugging)
        df["year"] = year
        df["file_source"] = fname
        
        frames.append(df)

print("\nConcatenating...")
df_all = pd.concat(frames, ignore_index=True)

# Sort nicely
df_all = df_all.sort_values(["timestamp", "detid_15"]).reset_index(drop=True)

print("Saving raw_traffic.csv ...")
df_all.to_csv(OUTPUT, index=False)

print(f"\nDONE. File saved to: {OUTPUT}")
print(f"Total rows: {len(df_all):,}")
print(f"Date range: {df_all.timestamp.min()} → {df_all.timestamp.max()}")
print(f"Unique detectors: {df_all.detid_15.nunique()}")


Aggregating detector data...

=== YEAR 2015 ===
=== YEAR 2016 ===
=== YEAR 2017 ===
=== YEAR 2018 ===
=== YEAR 2019 ===
=== YEAR 2020 ===
=== YEAR 2021 ===
=== YEAR 2022 ===
=== YEAR 2023 ===
=== YEAR 2024 ===

Concatenating...
Saving raw_traffic.csv ...

DONE. File saved to: detectors_data_raw\raw_traffic.csv
Total rows: 37,316,922
Date range: 2015-01-01 00:00:00 → 2024-12-31 23:00:00
Unique detectors: 591
